In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection  import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from tensorflow.contrib.layers import fully_connected


In [3]:
train_set = pd.read_csv('./temp/train_set_with_topic1.csv',encoding='gbk')
test_set = pd.read_csv('./temp/test_set_with_topic1.csv',encoding='gbk')

In [4]:
def resetIndex(df):
    vid = df['vid']
    df = df.drop('vid',axis=1)
    df = df.set_index(vid)
    return df

In [5]:
train_set = resetIndex(train_set)
test_set = resetIndex(test_set)

In [70]:
### construct NN



(26009, 86)

In [10]:
def train(train_data,col = ['收缩压','舒张压','血清甘油三酯','血清高密度脂蛋白','血清低密度脂蛋白'],learning_rate=0.01,n_epochs=2000):
    y = train_data[col]
    x = train_data.drop(col,axis = 1)
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=2018)
    print ("训练数据集样本数目：%d, 测试数据集样本数目：%d" % (x_train.shape[0], x_test.shape[0]))
    

    n_inputs = len(x_train.columns)
    n_hidden1 = 300
    n_hidden2 = 300
    n_hidden3 = 100
    n_outputs = 1
    
    X = tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
    y = tf.placeholder(tf.float32,shape=(None),name='y')
    
    def neuron_layer(X,n_neurons,name,activation = None):
        with tf.name_scope(name):
            n_inputs = int(X.get_shape()[1])
            stddev = 2 / np.sqrt(n_inputs)
            init = tf.truncated_normal((n_inputs,n_neurons),stddev=stddev)
            W = tf.Variable(init,name='Weights')
            b = tf.Variable(tf.zeros([n_neurons]),name='biaes')
            z = tf.matmul(X,W)+b
            if activation =='relu':
                return tf.nn.relu(z)
            else:
                return z
    
    with tf.name_scope("dnn"):
        hidden1 = neuron_layer(X,n_hidden1,"hidden1",activation="relu")
        hidden2 = neuron_layer(hidden1,n_hidden2,"hidden2",activation="relu")
        hidden3 = neuron_layer(hidden2,n_hidden3,"hidden3",activation="relu")
        y_ = neuron_layer(hidden3,n_outputs,"outputs")
    
    
    with tf.name_scope("loss"):
        #loss=tf.reduce_mean(tf.reduce_sum(tf.square(y-y_),reduction_indices=[1]),name="loss")
        loss = tf.reduce_mean(tf.square(y_-y),name='loss')
    

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()
 
    with tf.Session() as sess:
        init.run()
        for i in range(len(col)):
            print(col[i])
            for epoch in range(n_epochs):
                sess.run(training_op,feed_dict={X:x_train.values,y:y_train[col[i]].values})

                if epoch % 100:
                    #sess.run([loss],feed_dict={X:x_test.values,y:y_test[col[i]].values})
                    print("train_loss:  ",sess.run(loss,feed_dict={X:x_train.values,y:y_train[col[i]].values}))
                    print("valid_loss:   ",sess.run(loss,feed_dict={X:x_test.values,y:y_test[col[i]].values}))
            
            save_path = saver.save(sess,'./dnn_model{0}.ckpt'.format(i))

            

    

In [11]:
train(train_set)

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


训练数据集样本数目：20807, 测试数据集样本数目：5202
收缩压


KeyboardInterrupt: 